In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Este notebook require Tensorflow 2.X

In [2]:
print(tf.__version__)

2.1.0


### Descargar Extracto der Shakespeare 

In [3]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



### Set de caracteres en la obra (FYI):

In [4]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

### Tokenizacion del texto con Keras

In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

# probamos la palabra
tokenizer.texts_to_sequences(["Speak"])

[[8, 23, 2, 5, 25]]

In [6]:
# podemos converir esa secuencia en texto:
tokenizer.sequences_to_texts([[8, 23, 2, 5, 25]])

['s p e a k']

In [7]:
# numero de caracteres distintos
max_id = len(tokenizer.word_index) 

# cantidad total del caracteres
dataset_size = tokenizer.document_count

print("Elementos distintos",max_id)
print("Cantidad de elementos",dataset_size)

Elementos distintos 39
Cantidad de elementos 1115394


### Creamos el Train-Set usando las Funciones de NLP (ver Notebook #1)

In [8]:
# se convierte TODO el texto en secuencias.
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

# se genera un train-set del 90% de las secuencias
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [9]:
# se define que las cadenas son de 100 caracteres con 1 caracter de shift
n_steps = 100
window_length = n_steps + 1 # tamano de la ventana
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

# se generan las cadenas planas
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [10]:
# generamos el x_train y y_train usando el mismo codigo del notebook #1
np.random.seed(42)
tf.random.set_seed(42)
batch_size = 32

encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []

for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)

    dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda x_train, y_train: (tf.one_hot(x_train, depth=max_id), y_train))
dataset = dataset.prefetch(1)

# se revisan los tamanos de los tensores generados
for x, y in dataset.take(1):
    print(x.shape, y.shape)

(32, 100, 39) (32, 100)


In [11]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

### Modelo Stateful RNN-GRU y Entrenamiento

In [12]:
# arquitecturta
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation="softmax"))
])

# compilacion y entrenamiento
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                    callbacks=[ResetStatesCallback()])

Train for 313 steps
Epoch 1/50
313/313 [==============================] - 47s 149ms/step - loss: 2.6224
Epoch 2/50
313/313 [==============================] - 44s 139ms/step - loss: 2.2280
Epoch 3/50
313/313 [==============================] - 44s 140ms/step - loss: 2.1504
Epoch 4/50
313/313 [==============================] - 44s 140ms/step - loss: 2.4703
Epoch 5/50
313/313 [==============================] - 44s 140ms/step - loss: 2.3564
Epoch 6/50
313/313 [==============================] - 45s 142ms/step - loss: 2.2239
Epoch 7/50
313/313 [==============================] - 44s 140ms/step - loss: 2.0772
Epoch 8/50
313/313 [==============================] - 44s 140ms/step - loss: 2.0791
Epoch 9/50
313/313 [==============================] - 44s 140ms/step - loss: 2.0368
Epoch 10/50
313/313 [==============================] - 44s 140ms/step - loss: 1.9600
Epoch 11/50
313/313 [==============================] - 44s 140ms/step - loss: 1.9783
Epoch 12/50
313/313 [==============================] -

In [13]:
# esto es un super hack!
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())

In [14]:
model = stateless_model

# Guardar el Modelo
model.save('shakespeare-stateful.h5')